<a href="https://colab.research.google.com/github/keithy1012/MachineLearning/blob/main/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

class NeuralNetwork:
  def __init__(self, input_size, hidden_layers=3, layer_width = 5, learning_rate = 0.01, activation_function = "relu", output_size = 1):
    self.hidden_layers = hidden_layers
    self.layer_width = layer_width
    self.learning_rate = learning_rate
    self.activation_function = activation_function
    self.weights = [np.random.rand(input_size, layer_width)] + \
                       [np.random.rand(layer_width, layer_width) for _ in range(hidden_layers - 1)] + \
                       [np.random.rand(layer_width, output_size)]
    self.biases = [np.random.rand(layer_width) for _ in range(hidden_layers)] + [np.random.rand(output_size)]

  def relu(self, x):
      return np.maximum(0, x)

  def relu_derivative(self, x):
      return np.where(x > 0, 1, 0)

  def sigmoid(self, x):
      return 1 / (1 + np.exp(-x))

  def sigmoid_derivative(self, x):
      sig = self.sigmoid(x)
      return sig * (1 - sig)

  def tanh(self, x):
      return np.tanh(x)

  def tanh_derivative(self, x):
      return 1 - np.tanh(x)**2

  def activation(self, x):
      if self.activation_function == "relu":
            return self.relu(x)
      elif self.activation_function == "sigmoid":
            return self.sigmoid(x)
      else:
            return self.tanh(x)

  def activation_derivative(self, x):
      if self.activation_function == "relu":
            return self.relu_derivative(x)
      elif self.activation_function == "sigmoid":
            return self.sigmoid_derivative(x)
      else:
            return self.tanh_derivative(x)

  def forwardPass(self, inputs):
        self.layer_inputs = []  # Stores raw inputs to each layer
        self.layer_outputs = [inputs]  # Stores activated outputs of each layer

        # Forward pass through each hidden layer
        for i in range(self.hidden_layers):
            inputs = np.dot(inputs, self.weights[i]) + self.biases[i]
            self.layer_inputs.append(inputs)
            inputs = self.activation(inputs)
            self.layer_outputs.append(inputs)

        # Output layer forward pass (using linear activation for regression)
        final_input = np.dot(inputs, self.weights[-1]) + self.biases[-1]
        self.layer_inputs.append(final_input)
        output = final_input  # Linear output for regression
        self.layer_outputs.append(output)

        return output

  def loss(self, predictions, actual):
        return np.sum(np.square(predictions - actual)) / 2

  def backPropagation(self, X_train, y_train):
        predictions = self.forwardPass(X_train)
        output_error = predictions - y_train

        delta_output = output_error * self.sigmoid_derivative(self.layer_inputs[-1])
        weight_gradient = [np.dot(self.layer_outputs[-2].T, delta_output)]
        bias_gradient = [delta_output]

        delta = delta_output
        for i in range(self.hidden_layers - 1, -1, -1):
            layer_input = self.layer_inputs[i]
            delta = np.dot(delta, self.weights[i + 1].T) * self.activation_derivative(layer_input)
            weight_gradient.insert(0, np.dot(self.layer_outputs[i].T, delta))
            bias_gradient.insert(0, delta)

        for i in range(len(self.weights)):
            self.weights[i] -= self.learning_rate * weight_gradient[i]
            self.biases[i] -= self.learning_rate * np.sum(bias_gradient[i], axis=0)

        return self.loss(predictions, y_train)

In [ ]:
import numpy as np

input_size = 4
hidden_layers = 1
layer_width = 5
learning_rate = 1
output_size = 4

nn = NeuralNetwork(input_size = input_size, hidden_layers=hidden_layers, layer_width=layer_width, learning_rate=learning_rate, activation_function="sigmoid", output_size = output_size)

X_train = np.array([[1, 2, 3, 4]])
y_train = np.array([[2, 4, 6, 8]])

initial_predictions = nn.forwardPass(X_train)
initial_loss = nn.loss(initial_predictions, y_train)
print(f"Initial loss: {initial_loss}")

epochs = 1000
for epoch in range(epochs):
    loss = nn.backPropagation(X_train, y_train)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss}")

final_predictions = nn.forwardPass(X_train)
final_loss = nn.loss(final_predictions, y_train)
print(f"Final loss after training: {final_loss}")

print("Final predictions:", final_predictions)
print("True labels:", y_train)


Initial loss: 31.440779576020468
Epoch 0, Loss: 31.440779576020468
Epoch 100, Loss: 0.31485025972589764
Epoch 200, Loss: 0.14301886616206505
Epoch 300, Loss: 0.07562867103601476
Epoch 400, Loss: 0.043008152109510825
Epoch 500, Loss: 0.025553862485779438
Epoch 600, Loss: 0.015633866332890076
Epoch 700, Loss: 0.009764289736424186
Epoch 800, Loss: 0.0061912120658494385
Epoch 900, Loss: 0.003970426830773674
Final loss after training: 0.0025684330952177397
Final predictions: [[2.         4.00000005 6.00000056 7.92832807]]
True labels: [[2 4 6 8]]
